In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import math
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input director
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [60]:
# Load data khi lấy từ lớp full connected
features_train = np.load("/kaggle/input/ge-features/features_train.npy")

In [61]:
features_train.shape

(8502, 6400)

In [62]:
# # Merge train and valid's label
train_label_df = pd.read_csv("/kaggle/input/ge-muse-split/train_label.csv")
valid_label_df = pd.read_csv("/kaggle/input/ge-muse-split/val_label.csv")
label_df = pd.concat((train_label_df, valid_label_df), axis = 0)
label_df.shape

(8502, 7)

In [63]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()

label_pca = label_df["Rhythm"]
label_enc = lb.fit_transform(label_pca)
label_enc

array([0, 1, 1, ..., 2, 1, 2])

In [74]:
y_test = lb.fit_transform(pd.read_csv("/kaggle/input/ge-muse-split/test_label.csv")["Rhythm"].values)
y_test.shape

(2126,)

In [81]:
X_train = features_train
y_train = label_enc
X_test = np.load("/kaggle/input/ge-features/features_test.npy")

clf = DecisionTreeClassifier()
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter':['best', 'random'],
    'max_features':['auto','sqrt','log2'],
}

grid = GridSearchCV(clf, param_grid, refit = True, verbose = 3) 
  
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END criterion=gini, max_features=auto, splitter=best;, score=0.949 total time=   0.8s
[CV 2/5] END criterion=gini, max_features=auto, splitter=best;, score=0.941 total time=   0.8s
[CV 3/5] END criterion=gini, max_features=auto, splitter=best;, score=0.942 total time=   0.7s
[CV 4/5] END criterion=gini, max_features=auto, splitter=best;, score=0.942 total time=   0.8s
[CV 5/5] END criterion=gini, max_features=auto, splitter=best;, score=0.899 total time=   0.6s
[CV 1/5] END criterion=gini, max_features=auto, splitter=random;, score=0.941 total time=   0.2s
[CV 2/5] END criterion=gini, max_features=auto, splitter=random;, score=0.941 total time=   0.2s
[CV 3/5] END criterion=gini, max_features=auto, splitter=random;, score=0.934 total time=   0.2s
[CV 4/5] END criterion=gini, max_features=auto, splitter=random;, score=0.939 total time=   0.2s
[CV 5/5] END criterion=gini, max_features=auto, splitter=random;, score=0.89

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             verbose=3)

In [82]:
print(grid.best_estimator_)
print(grid.best_params_)

DecisionTreeClassifier(criterion='log_loss', max_features='sqrt')
{'criterion': 'log_loss', 'max_features': 'sqrt', 'splitter': 'best'}


In [83]:
y_pred = grid.predict(X_test)

In [84]:
y_test

array([0, 3, 0, ..., 0, 0, 2])

In [85]:
from sklearn.metrics import classification_report

print("Kết quả không áp dụng PCA:")
print(classification_report(y_test, y_pred, target_names= lb.classes_))

result = classification_report(y_test, y_pred, target_names= lb.classes_, output_dict = True)
result_df = pd.DataFrame(result).transpose()
result_df

result_df.to_csv('./result.csv', index=True)

Kết quả không áp dụng PCA:
              precision    recall  f1-score   support

        AFIB       0.86      0.92      0.89       444
        GSVT       0.88      0.84      0.86       460
          SB       0.96      0.97      0.96       778
          SR       0.93      0.89      0.91       444

    accuracy                           0.91      2126
   macro avg       0.91      0.90      0.90      2126
weighted avg       0.91      0.91      0.91      2126

